## Numerical Analysis - Fall semester 2024

# Serie 08 - Numerical integration and linear systems

Importing some useful packages.

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import scipy as sp
import timeit

<hr style="clear:both">

### Adaptive quadrature


We would like to approximate the integral $I(f)=\int_a^bf(x)~\mathrm{d}x$
within a given tolerance $tol$. That is, we want to determine $h$ such as 

$$
|I(f)-Q_h(f)|\le tol,
$$
where $Q_h$ is a quadrature formula of order $p$. As the exact integral $I(f)$ is, in general, unknown, we have to introduce a suitable error estimator. A possible estimator of the error for $Q_{h}(f)$ is

$$
\eta:=\frac{|Q_{h}(f)-Q_{2h}(f)|}{2^p-1},
$$

which uses Richardson's extrapolation. It is based on the fact that

$$
\tilde{Q}_{h}:=\frac{2^pQ_{h}(f)-Q_{2h}(f)}{2^p-1}
$$

is an approximation of $I(f)$ of order (at least) $p+1$, as long as $f$ is regular enough, and therefore

$$
\underbrace{|I(f)-Q_{h}(f)|}_{\mathcal{O}(h^p)}\leq \underbrace{|I(f)-\tilde{Q}_{h}(f)|}_{\mathcal{O}(h^{p+1})}+\underbrace{|\tilde{Q}_{h}(f)-Q_{h}(f)|}_{\mathcal{O}(h^p)}\approx |\tilde{Q}_{h}(f)-Q_{h}(f)|=\eta.
$$

To solve the following exercises, we provide you with the implementation of the midpoint quadrature, which you've already used to solve last week's notebook. Run the below cell to use it later on.

In [ ]:
def midpoint(a, b, n, f):
    # Composite midpoint formula
    #  - a,b: boundaries of the integration interval
    #  - n: number of sub-intervals
    #  - f: function to integrate
    h = (b - a) / n
    xi = np.linspace(a + h / 2, b - h / 2, n)  # quadrature nodes
    alphai = np.full(n, h)  # weights
    Qh_mp = np.dot(alphai, f(xi))  # quadrature formula
    return Qh_mp

<div class="alert alert-success">
    
**Exercise 1:** Implement the function `adaptive_quadrature`, which adaptively approximates the integral $I(f)=\int_a^b f(x)~\mathrm{d}x$ using the midpoint quadrature implemented in the `midpoint` function above. That is, it iteratively doubles the number of sub-intervals `n` of the composite midpoint quadrature, computes the corresponding midpoint quadrature, and computes the error estimate $\eta$, until the error estimate $\eta$ lies below the tolerance.
</div>

In [ ]:
def adaptive_quadrature(a, b, f, tol):
    n = 1  # current number of sub-intervals
    p = 2  # order of midpoint formula

    # lists which should be filled in at every iteration
    Qh_list = [midpoint(a, b, n, f)]  # quadrature approximation for each number of sub-intervals
    h_list = [(b - a) / n]  # h for each number of sub-intervals
    eta = tol + 1  # set the first error estimate to not already meet tolerance

    while eta > tol:
        ### BEGIN SOLUTION
        n = 2 * n  # double the number of sub-intervals
        Qh_list.append(midpoint(a, b, n, f))  # compute midpoint quadrature
        h_list.append((b - a) / n)  # re-compute h
        eta = abs(Qh_list[-1] - Qh_list[-2]) / (2 ** p - 1)  # error estimate
        ### END SOLUTION

    # convert lists to numpy arrays for easier manipulations
    return np.array(Qh_list), np.array(h_list)

<div class="alert alert-success">
    
**Exercise 2:** 
Consider the function $f(x)=\frac{\sin(x)\cos^3(x)}{4-\cos^2(2x)}$, on the interval $[a,b]=[0,\pi/2]$. Using the adaptive quadrature you have just implemented based on the error estimate $\eta$, determine (numerically) the number $n=(b-a)/h$ of sub-intervals necessary to obtain an approximation of the integral with an estimated error lower than $10^{-5}$.
</div>


In [ ]:
def f(x):
    ### BEGIN SOLUTION
    return np.sin(x) * np.cos(x) ** 3 / (4 - np.cos(2 * x) ** 2)
    ### END SOLUTION

### BEGIN SOLUTION
p = 2
tol = 1e-5
a = 0
b = np.pi / 2

Qh_list, h_list = adaptive_quadrature(a, b, f, tol)

print("number of sub-intervals needed: {}".format(int((b - a) / h_list[-1])))
### END SOLUTION

<div class="alert alert-success">
    
**Exercise 3:** 
Knowing that the exact value of the integral is $I(f)=\log(3)/16$, plot the graph of the estimator and of the "true" error in terms of $h$ in logarithmic scale (command `plt.loglog`). What can you say about the order of convergence and the quality of the error estimator?
</div>


In [ ]:
I_f = np.log(3) / 16

### BEGIN SOLUTION
error = np.abs(I_f - Qh_list)
error_estimate_eta = np.abs(Qh_list[1:] - Qh_list[:-1]) / (2 ** p - 1)

plt.figure()
plt.loglog(h_list[1:], error[1:], label=r"errors $|I(f) - Q_h|$")
plt.loglog(h_list[1:], error_estimate_eta, label=r"error estimates $\eta$")
plt.loglog(h_list[1:], h_list[1:], ls="--", color="black", label=r"$h$")
plt.loglog(h_list[1:], h_list[1:] ** 2, ls=":", color="black", label=r"$h^2$")
plt.grid(True, which="major", ls="-")
plt.grid(True, which="minor", ls="--")
plt.xlabel(r"$h$")
plt.legend()
plt.show()

# The error as well as the estimator based on Richardson's extrapolation
# are of order 2 in terms of h. Moreover, the plot confirms that the estimator
# is a good estimator of the error: indeed both curves are almost identical.
### END SOLUTION

<div class="alert alert-success">
    
**Exercise 4:** 
On the same graph as in Exercise 3, plot the error $|I(f) - \tilde{Q}_h|$ in terms of $h$, where $\tilde{Q}_h$ is the Richardson's extrapolation based on $Q_h$ and $Q_{2h}$. What order of convergence do you observe in this case?
</div>


In [ ]:
### BEGIN SOLUTION
Qh_tilde_list = (2 ** p * Qh_list[1:] - Qh_list[:-1]) / (2 ** p - 1)
error_richardson = np.abs(I_f - Qh_tilde_list)

plt.figure()
plt.loglog(h_list[1:], error[1:], label=r"errors $|I(f)- Q_h|$")
plt.loglog(h_list[1:], error_estimate_eta, label=r"error estimates $\eta$")
plt.loglog(h_list[1:], error_richardson, label=r"errors $|I(f) - \tilde{Q}_h|$")
plt.loglog(h_list[1:], h_list[1:] ** 2, ls="--", color="black", label=r"$h^2$")
plt.loglog(h_list[1:], h_list[1:] ** 4, ls=":", color="black", label=r"$h^4$")
plt.grid(True, which="major", ls="-")
plt.grid(True, which="minor", ls="--")
plt.xlabel(r"$h$")
plt.legend()
plt.show()

# We see that Qh_tilde(f) is a higher order approximation of I(f)
# as the error |I(f) - Qh_tilde(f)| in logarithmic scale decreases
# with slope 4. Note that we got an approximation of order 4 and not 2.
### END SOLUTION

<div class="alert alert-success">
    
**Exercise 5:** 
Repeat all the previous exercises in this section for $f(x)=\sqrt{|x|^3}$ and $[a,b]=[-2,2]$, the exact value of the integral being in this case $I(f)=16\sqrt{2}/5$. Comment the obtained results.
</div>


In [ ]:
I_f = 16 * np.sqrt(2) / 5

### BEGIN SOLUTION
def f(x):
    return np.sqrt(np.abs(x) ** 3)

a = -2
b = 2

Qh_list, h_list = adaptive_quadrature(a, b, f, tol)
print("number of sub-intervals needed: {}".format(int((b - a) / h_list[-1])))

error = np.abs(I_f - Qh_list)
error_estimate_eta = np.abs(Qh_list[1:] - Qh_list[:-1]) / (2 ** p - 1)

plt.figure()
plt.loglog(h_list[1:], error[1:], label=r"errors $|I(f) - Q_h|$")
plt.loglog(h_list[1:], error_estimate_eta, label=r"error estimates $\eta$")
plt.loglog(h_list[1:], h_list[1:], ls="--", color="black", label=r"$h$")
plt.loglog(h_list[1:], h_list[1:] ** 2, ls=":", color="black", label=r"$h^2$")
plt.grid(True, which="major", ls="-")
plt.grid(True, which="minor", ls="--")
plt.xlabel(r"$h$")
plt.legend()
plt.show()

Qh_tilde_list = (2 ** p * Qh_list[1:] - Qh_list[:-1]) / (2 ** p - 1)
error_richardson = np.abs(I_f - Qh_tilde_list)

plt.figure()
plt.loglog(h_list[1:], error[1:], label=r"errors $|I(f)- Q_h|$")
plt.loglog(h_list[1:], error_estimate_eta, label=r"error estimates $\eta$")
plt.loglog(h_list[1:], error_richardson, label=r"errors $|I(f) - \tilde{Q}_h|$")
plt.loglog(h_list[1:], h_list[1:] ** 2, ls="--", color="black", label=r"$h^2$")
plt.loglog(h_list[1:], h_list[1:] ** 4, ls=":", color="black", label=r"$h^4$")
plt.grid(True, which="major", ls="-")
plt.grid(True, which="minor", ls="--")
plt.xlabel(r"$h$")
plt.legend()
plt.show()

# In this case we need M=1024 sub-intervals to achieve the desired tolerance,
# which corresponds to h=1/256. 
# In this case, Qh_tilde(f) is an approximation with an order smaller than 4.
# This is due to the lack of regularity of the function f (see last week's notebook).

### END SOLUTION

<div class="alert alert-success">
    
**Exercise 6:** 
Repeat all the previous exercises in this section for $f(x)=\log(x)$ and $[a,b]=[0,1]$, the exact value of the integral being in this case $I(f)=-1$. Comment the obtained results.
</div>

In [ ]:
I_f = -1

### BEGIN SOLUTION
def f(x):
    return np.log(x)

a = 0
b = 1

Qh_list, h_list = adaptive_quadrature(a, b, f, tol)
print("number of sub-intervals needed: {}".format(int((b - a) / h_list[-1])))

error = np.abs(I_f - Qh_list)
error_estimate_eta = np.abs(Qh_list[1:] - Qh_list[:-1]) / (2 ** p - 1)
ratio_estimate_to_error = error_estimate_eta / error[1:]
print("ratios of estimate to errors: {}".format(ratio_estimate_to_error))

plt.figure()
plt.loglog(h_list[1:], error[1:], label=r"errors $|I(f) - Q_h|$")
plt.loglog(h_list[1:], error_estimate_eta, label=r"error estimates $\eta$")
plt.loglog(h_list[1:], h_list[1:], ls="--", color="black", label=r"$h$")
plt.grid(True, which="major", ls="-")
plt.grid(True, which="minor", ls="--")
plt.xlabel(r"$h$")
plt.legend()
plt.show()

Qh_tilde_list = (2 ** p * Qh_list[1:] - Qh_list[:-1]) / (2 ** p - 1)
error_richardson = np.abs(I_f - Qh_tilde_list)

plt.figure()
plt.loglog(h_list[1:], error[1:], label=r"errors $|I(f)- Q_h|$")
plt.loglog(h_list[1:], error_estimate_eta, label=r"error estimates $\eta$")
plt.loglog(h_list[1:], error_richardson, label=r"errors $|I(f) - \tilde{Q}_h|$")
plt.loglog(h_list[1:], h_list[1:], ls="--", color="black", label=r"$h$")
plt.grid(True, which="major", ls="-")
plt.grid(True, which="minor", ls="--")
plt.xlabel(r"$h$")
plt.legend()
plt.show()

# The stopping criterion of the *while* loop is reached for M=16384 sub-intervals, which corresponds
# to h=1/16384. 
# In this case, Qh(f) and Qh_tilde(f) are approximations of order 1-ε, with ε small, due to the fact
# that the function f is not continuous (discontinuous in x=0).
# Moreover, even though the order of convergence of the estimator is similar to the one of the error,
# the estimator underestimates the error by a factor 3 and is therefore
# not reliable. This explains why even though the obtained error for M=1/16384 does not satisfy the
# desired tolerance, the stopping criterion of the adaptive loop (which is based on the estimator)
# was met.
### END SOLUTION

<hr style="clear:both">

### Simple linear systems

Consider the following two linear systems:

$$
  A\mathbf{x} = \mathbf{b}_1, \qquad \text{and} \qquad
  A= \begin{pmatrix}
          2 & 4 & 8 \\
          1 & 1 & 4 \\
          3 & 6 & 7
        \end{pmatrix}, \quad
  \mathbf{b}_1 = \begin{pmatrix} 6 \\ 5 \\ 4 \end{pmatrix},
$$

and

$$
  B\mathbf{x} = \mathbf{b}_2, \qquad \text{and} \qquad
  B = \begin{pmatrix}
          1 & 1 & 1 & 1 \\
          2 & 2 & 5 & 3 \\
          4 & 6 & 8 & 0 \\
          3 & 3 & 9 & 8
        \end{pmatrix}, \quad
  \mathbf{b}_2 = \begin{pmatrix} 1 \\ 2 \\ 5 \\ 0 \end{pmatrix}.
$$

<div class="alert alert-success">
    
**Exercise 7 (Theoretical):** Compute the LU factorization of the matrix $A$ by hand.
</div>

=== BEGIN MARK SCHEME ===

Let us consider the given matrix, starting from the first step of the
Gauss method (LU factorization):

$$
A^{(1)} = A = \left(
  \begin{array}{ccc}
  2 & 4 & 8 \\
  1 & 1 & 4 \\
  3 & 6 & 7 \\
  \end{array}
  \right) =
  \left(
   \begin{array}{ccc}
          a_{11}^{(1)} & a_{12}^{(1)} & a_{13}^{(1)} \\
          a_{21}^{(1)} & a_{22}^{(1)} & a_{23}^{(1)} \\
          a_{31}^{(1)} & a_{32}^{(1)} & a_{33}^{(1)}
   \end{array} \right) \, .   
$$

We compute the multipliers

$$
l_{21} = \frac{a_{21}^{(1)}}{a_{11}^{(1)}} = \frac{1}{2} \quad \textrm{and} \quad
l_{31} = \frac{a_{31}^{(1)}}{a_{11}^{(1)}} = \frac{3}{2} \, ,
$$

and we define

\begin{align*}
a_{22}^{(2)} &=& a_{22}^{(1)}& - l_{21} a_{12}^{(1)} = -1 \\
a_{23}^{(2)} &=& a_{23}^{(1)}& - l_{21} a_{13}^{(1)} =  0 \\
a_{32}^{(2)} &=& a_{32}^{(1)}& - l_{31} a_{12}^{(1)} =  0 \\
a_{33}^{(2)} &=& a_{33}^{(1)}& - l_{31} a_{13}^{(1)} = -5.
\end{align*}

We therefore get the matrix

$$
A^{(2)} = \left(
  \begin{array}{ccc}
  2 & 4 & 8 \\
  0 & -1 & 0 \\
  0 & 0 & -5 \\
  \end{array}
  \right) \, 
$$

which is upper triangular. We deduce that $l_{31}=0$ and we finally get (let us remind that the diagonal elements of $L$ all equal to $1$)

$$
  L = \begin{pmatrix}
        1 & 0 & 0 \\
        1/2 & 1 & 0 \\
        3/2 & 0 & 1
      \end{pmatrix} \quad \text{and} \quad
  U = \begin{pmatrix}
        2 & 4 & 8 \\
        0 & -1 & 0 \\
        0 & 0 & -5
      \end{pmatrix}.
$$

=== END MARK SCHEME ===

<div class="alert alert-success">
    
**Exercise 8 (Theoretical):** Solve the linear system $A \mathbf{x} = \mathbf{b}_1$ using the factorization found in the previous exercise.
</div>

=== BEGIN MARK SCHEME ===

We first solve $L\mathbf{y} = \mathbf{b}_1$ and then $U\mathbf{x} = \mathbf{y}$.

* first system (forward substitution):

$$
  \begin{cases}
    y_1 = 6 \\[2ex]
    \dfrac{1}{2}y_1 + y_2 = 5 \\[2ex]
    \dfrac{3}{2}y_1 + y_3 = 4
  \end{cases} \quad \Longrightarrow \quad
  \begin{array}{l}
   y_1 = 6 \\[2ex]
   y_2 = 5 - \dfrac{1}{2}y_1 = 2 \\[2ex]
   y_3 = 4 - \dfrac{3}{2}y_1 = -5
  \end{array}
$$

* second system (backward substitution):

$$
  \begin{cases}
    2x_1 + 4x_2 + 8x_3 = 6 \\
    -1x_2 = 2 \\
    -5x_3 = -5
  \end{cases} \quad \Longrightarrow \quad
  \begin{array}{l}
    x_3 = 1 \\
    x_2 = -2 \\
    x_1 = \dfrac{1}{2}(6 - 4x_2 - 8x_3) = 3
  \end{array}
$$

=== END MARK SCHEME ===

<div class="alert alert-success">
    
**Exercise 9 (Theoretical):** Check that the algorithm of the LU factorization without pivoting for the matrix $B$ cannot be executed until the end.
</div>

=== BEGIN MARK SCHEME ===

We can check that the algorithm without pivoting cannot be applied using the Gauss elimination method. We have:

$$
B^{(1)}=B=\begin{pmatrix}
          1 & 1 & 1 & 1 \\
          2 & 2 & 5 & 3 \\
          4 & 6 & 8 & 0 \\
          3 & 3 & 9 & 8
        \end{pmatrix}
$$

$$
B^{(2)}=\begin{pmatrix}
          1 & 1 & 1 & 1 \\
          0 & 0 & 3 & 1 \\
          0 & 2 & 4 & -4 \\
          0 & 0 & 6 & 5
        \end{pmatrix}
$$

therefore, we have to interrupt the Gauss method without pivoting at the second step as $b_{22}^{(2)}=0$.

=== END MARK SCHEME ===


<div class="alert alert-success">
    
**Exercise 10 (Theoretical):** Find a permutation matrix $P$ by rows such as $PB$ is factorisable, then calculate the LU factorization of $PB$.
</div>

=== BEGIN MARK SCHEME ===

We just have to swap in the matrix $B$ the second line with another one, for example the third, in order to be able to continue the  factorization algorithm from the previous point. If we take

$$
  P=\begin{pmatrix}
       1 & 0 & 0 & 0 \\
       0 & 0 & 1 & 0 \\
       0 & 1 & 0 & 0 \\
       0 & 0 & 0 & 1
    \end{pmatrix} \qquad \implies \qquad
 PB = \begin{pmatrix}
          1 & 1 & 1 & 1 \\
          4 & 6 & 8 & 0 \\
          2 & 2 & 5 & 3 \\
          3 & 3 & 9 & 8
        \end{pmatrix},
$$

we get to the following factorization

$$
  L = \begin{pmatrix}
          1 & 0& 0& 0 \\
          4 & 1 & 0& 0\\
          2 & 0 & 1 & 0\\
          3 & 0 & 2 & 1
        \end{pmatrix}, \qquad
  U = \begin{pmatrix}
          1 & 1 & 1 & 1 \\
           0 & 2 & 4 &-4 \\
           0 &  0 & 3 & 1 \\
           0 & 0  & 0  & 3
       \end{pmatrix}.
$$

We immediately check that $LU=PB$.

=== END MARK SCHEME ===


<div class="alert alert-success">
    
**Exercise 11 (Theoretical):** Solve the linear system $B\mathbf{x} = \mathbf{b}_2$ using the factorization previously found.
</div>

=== BEGIN MARK SCHEME ===

We have to solve the systems $L \mathbf{y} = P\mathbf{b}_2$ and $U \mathbf{x} = \mathbf{y}$. We find

$$
  \mathbf{y} = \left(1, 1, 0, -3\right)^\top, \qquad \mathbf{x} = \left(\dfrac{23}{6},
  -\dfrac{13}{6}, \dfrac{1}{3}, -1\right)^\top.
$$

=== END MARK SCHEME ===

<div class="alert alert-success">
    
**Exercise 12 (Theoretical):** Compute the determinant of the matrix $B$ using its LU factorization.

*Hint:* We know that
$$
\text{det}(B)=\text{det}(P^{-1}LU)=\displaystyle\frac{\text{det}(L) \text{det}(U)}{\text{det}(P)}.
$$
</div>

=== BEGIN MARK SCHEME ===

We use the relation $\text{det}(B)=\text{det}(P^{-1}LU)=\displaystyle\dfrac{1}{\text{det}(P)}\text{det}(L) \text{det}(U)$. Noticing 
that the determinant of a triangular matrix is given by 
the product of its diagonal elements, we find that

$$
  \textrm{det}(B) = -1 \cdot 1 \cdot 18 = -18 .
$$

Additionally, note that the determinant of a permutation matrix between two
rows is always $-1$, and therefore the determinant of $P$ is $+1$
if we performed an even number of permutations and
$-1$ if we performed an odd number of permutations:
in the case of this exercise, we have $\text{det}(P)=-1$.

=== END MARK SCHEME ===

<hr style="clear:both">

### LU factorization for dense and sparse matrix 

<div class="alert alert-success">
    
**Exercise 13:** Complete the function `binomial_matrix` which creates a matrix $A \in \mathbb{R}^{m \times m}$ whose entries $a_{ij}$ for $i,j = 1, 2, \dots, m$ independently follow a binomial distribution with $n \in \mathbb{N}$ trials and a success probability of $p \in [0, 1]$. Use your implementation to create the $(n=10, p=0.5)$-binomial matrix $A \in \mathbb{R}^{400 \times 400}$. Visualize its non-zero entries using the function `plt.spy(A)`, which plots the non-zero entries of a matrix in black.

*Hint:* The NumPy function `np.random.binomial(n, p, (m, m))` outputs an $m \times m$ NumPy array whose entries are independently $(n, p)$-binomial distributed. The seed of NumPy's random number generator is fixed to `seed=0`, such that you will always get the same random matrix for the same parameters.
</div>

In [ ]:
def binomial_matrix(m, n, p, seed=0):
    np.random.seed(seed)
    ### BEGIN SOLUTION
    A = np.random.binomial(n, p, (m, m))
    return A
    ### END SOLUTION

### BEGIN SOLUTION
n = 10
p = 0.5
A = binomial_matrix(400, n, p)
plt.spy(A)
plt.show()
### END SOLUTION

<div class="alert alert-success">
    
**Exercise 14:** Compute the LU factorization using the function `sp.linalg.lu`, which takes as an input a matrix $A$ and returns the three matrices $P$, $L$, and $U$ which characterize the LU factorization of $A$. Visualize the non-zero entries of $L$, $U$, and $P$ (again use `plt.spy`). Have row-permutations been performed? How do the number of non-zero elements in $L$ and $U$ compare with what we saw for $A$?
</div>

In [ ]:
### BEGIN SOLUTION
P, L, U = sp.linalg.lu(A)

fig, ax = plt.subplots(1, 3, figsize=(10, 3))
ax[0].spy(P)
ax[0].set_title("$P$")
ax[1].spy(L)
ax[1].set_title("$L$")
ax[2].spy(U)
ax[2].set_title("$U$")
plt.show()

# We notice that the matrix P is not the identity matrix, this means that we can be sure
# that at least one permutation was performed. We see that A has almost only non-zero elements
# whereas in L and U, only around half of the elements are non-zero.
# The non-zero structure of L and U displays the triangular patterns typical of LU factorization. 

### END SOLUTION

<div class="alert alert-success">
    
**Exercise 15:** For the binomial matrix $A$ with dimensions $m=20^2, 21^2,\ldots,35^2$, compute the number of non-zero entries $\text{nnz}(A)$ in $A$ and compare it to the number of non-zero elements $\text{nnz}(L) + \text{nnz}(U)$ in its LU factors. Plot the results in a logarithmic plot, and add the number of entries $m^2$ of the full matrix for comparison.

*Hint:* Use `np.count_nonzero` to count the number of non-zero entries in a NumPy array.
</div>

In [ ]:
### BEGIN SOLUTION
m_list = np.arange(20, 35) ** 2

nnz_list_A = []
nnz_list_A_LU = []
for m in m_list:
    A = binomial_matrix(m, n, p)
    P, L, U = sp.linalg.lu(A)
    nnz_list_A.append(np.count_nonzero(A))
    nnz_list_A_LU.append(np.count_nonzero(L) + np.count_nonzero(U))

plt.loglog(m_list, nnz_list_A, label="non-zero elements in $A$")
plt.loglog(m_list, nnz_list_A_LU, label="non-zero elements in $L$ and $U$")
plt.loglog(m_list, m_list**2, ls="--", c="k", label="$m^2$")
plt.ylabel("non-zero elements")
plt.xlabel("matrix dimension $m$")
plt.grid(True, which="major", linestyle="-")
plt.grid(True, which="minor", linestyle="--")
plt.legend()
plt.show()

# Since A is almost a dense matrix, the number of non-zero elements are very close to m².
# Similarly, the number of non-zero elements of both the LU factors together are also approximately m².
### END SOLUTION

Below, we provide you with a function which computes the runtime of the LU factorization algorithm `sp.linalg.lu` for any matrix `A` by executing the function on the matrix multiple times, and averaging the individual runtimes. The number of repetitions of the function execution can be controlled with the argument `repeats`.

In [ ]:
def lu_runtime(A, repeats):
    t = timeit.timeit(lambda: sp.linalg.lu(A), number=repeats) / repeats
    return t

<div class="alert alert-success">
    
**Exercise 16:** Measure the approximate runtime for computing the LU factorization of the binomial matrix $A$ of dimensions $m=20^2, 21^2,\ldots,35^2$. Set `repeats=50` to average over $50$ executions of the algorithm to get a more stable estimate of the true run-time.
</div>

<div class="alert alert-info">

**Note:** It may happen that the runtime of the first $m$ is significantly higher. This is due to the caching which Python performes in the background. If it bothers you, you may simply ignore the first $m$.
</div>

In [ ]:
### BEGIN SOLUTION
t_list_A = []
for m in m_list:
    A = binomial_matrix(m, n, p)
    t = lu_runtime(A, repeats=50)
    t_list_A.append(t)

plt.loglog(m_list, t_list_A, label="LU factorization of $A$")
plt.plot(m_list, 1e-8 * m_list ** 2, ls="--", c="k", label="$m^2$")
plt.ylabel("runtime $t$")
plt.xlabel("matrix dimension $m$")
plt.grid(True, which="major", linestyle="-")
plt.grid(True, which="minor", linestyle="--")
plt.legend()
plt.show()

# We notice that the run-time grows slightly stronger than m²
### END SOLUTION

The following function `laplacian_matrix` generates a Laplacian matrix $B \in \mathbb{R}^{m \times m}$.

In [ ]:
def laplacian_matrix(m):
    n = int(np.sqrt(m))
    d = np.ones(n ** 2)
    mat = sp.sparse.spdiags([d, -2 * d, d], [-1, 0, 1], n, n)
    I = sp.sparse.eye(n)
    A = sp.sparse.kron(I, mat) + sp.sparse.kron(mat, I) 
    return A.todense()

<div class="alert alert-success">
    
**Exercise 17:** Repeat all of the above exercises in this section for the Laplacian matrix $B$ with the same dimensions. Compare to the results you've obtained for the binomial matrix $A$.
</div>

In [ ]:
### BEGIN SOLUTION
B = laplacian_matrix(400)
plt.spy(B)
plt.show()

P, L, U = sp.linalg.lu(B)

fig, ax = plt.subplots(1, 3, figsize=(10, 3))
ax[0].spy(P)
ax[0].set_title("$P$")
ax[1].spy(L)
ax[1].set_title("$L$")
ax[2].spy(U)
ax[2].set_title("$U$")
plt.show()

nnz_list_B = []
nnz_list_B_LU = []
for m in m_list:
    B = laplacian_matrix(m)
    P, L, U = sp.linalg.lu(B)
    nnz_list_B.append(np.count_nonzero(B))
    nnz_list_B_LU.append(np.count_nonzero(L) + np.count_nonzero(U))

plt.loglog(m_list, nnz_list_B, label="non-zero elements in $B$")
plt.loglog(m_list, nnz_list_B_LU, label="non-zero elements in $L$ and $U$")
plt.loglog(m_list, m_list ** (3/2), ls="--", c="k", label="$m^{3/2}$")
plt.ylabel("non-zero elements")
plt.xlabel("matrix dimension $m$")
plt.grid(True, which="major", linestyle="-")
plt.grid(True, which="minor", linestyle="--")
plt.legend()
plt.show()

t_list_B = []
for m in m_list:
    B = laplacian_matrix(m)
    t = lu_runtime(B, repeats=50)
    t_list_B.append(t)

plt.loglog(m_list, t_list_B, label="LU factorization of $B$")
plt.plot(m_list, 1e-8 * m_list ** 2, ls="--", c="k", label="$m^2$")
plt.ylabel("runtime $t$")
plt.xlabel("matrix dimension $m$")
plt.grid(True, which="major", linestyle="-")
plt.grid(True, which="minor", linestyle="--")
plt.legend()
plt.show()

# The LU factorization of a sparse matrix A often results in L and U matrices that are denser
# than A. This illustrates how factorization affects matrix sparsity, which is especially relevant
# for applications where memory and computational efficiency are key considerations.
# We notice that the computing time, which increases as m³ for the binomial case, in this case
# only increases proportionally to m². The memory consumption, which increases as m² for the
# binomial case, is here proportional to m^(3/2). This is due to the fact that the LU factorization
# of A will fill the band of the matrix, which has a width of √m. The matrices L and U therefore
# contain O(m√m) = O(m^(3/2)) non-zero elements, hence the factorization needs O(m√m²) = O(m²) operations.
### END SOLUTION

<hr style="clear:both">

## The end

Congratulations! This is the end of the eighth exercise notebook.